## All COVID-19 Cases by Country

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Specific to JHCSSE data
def clean_data(file):
    df = pd.read_csv(file)
    df = df.drop(['Lat','Long','Province/State'], axis=1)
    df.rename(columns={'Country/Region': 'Country_Region'}, inplace = True)
    df = df.groupby('Country_Region').sum()
    df.rename(index={'US':'United States',
        'Taiwan*':'Taiwan',
        'Korea, South':'South Korea',
        'Congo (Brazzaville)':'Congo',
        'Congo (Kinshasa)':'Democratic Republic of Congo',
        'Czechia':'Czech Republic',
        'Gambia, The':'Gambia',},inplace=True)
    return df

#### Confirmed Cases

In [3]:
# All Countries
confirmed_df = clean_data('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
confirmed_df.head(1)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,840,906,933,996,1026,1092,1176,1279,1351,1463


In [4]:
# Global row
global_row = pd.Series(confirmed_df.sum(), name='Global')
confirmed_df = confirmed_df.append(global_row)
confirmed_df.tail()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20
Country_Region,,,,,,,,,,,,,,,,,,,,,
Western Sahara,0,0,0,0,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
Yemen,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Zambia,0,0,0,0,0,0,0,0,0,0,...,48,52,57,61,65,70,74,76,84,84
Zimbabwe,0,0,0,0,0,0,0,0,0,0,...,23,24,25,25,25,28,28,28,29,31
Global,555,654,941,1434,2118,2927,5578,6166,8234,9927,...,2151421,2239144,2316716,2400078,2471247,2548091,2623049,2707728,2809499,2896744


In [5]:
# Top Countries
n = 9
last_day = confirmed_df.columns[-1]
top_confirmed_df = confirmed_df.nlargest(n, last_day)
top_confirmed_df.drop(index='Global', inplace=True)
top_confirmed_df.index

Index(['United States', 'Spain', 'Italy', 'France', 'Germany',
       'United Kingdom', 'Turkey', 'Iran'],
      dtype='object', name='Country_Region')

#### Fatalities

In [6]:
# All Countries
fatalities_df = clean_data('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

# Global row
global_row = pd.Series(fatalities_df.sum(), name='Global')
fatalities_df = fatalities_df.append(global_row)

# Top Countries
top_fatalities_df = fatalities_df.loc[fatalities_df.index.isin(top_confirmed_df.index)]

#### Recovered

In [7]:
# All Countries
recovered_df = clean_data('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

# Global row
global_row = pd.Series(recovered_df.sum(), name='Global')
recovered_df = recovered_df.append(global_row)

# Top Countries
top_recovered_df = recovered_df.loc[recovered_df.index.isin(top_confirmed_df.index)]

#### Infected = Confirmed - Recovered

In [8]:
# All Countries
infected_df = confirmed_df-recovered_df

# Global row
global_row = pd.Series(infected_df.sum(), name='Global')
infected_df = infected_df.append(global_row)

# Top Countries
top_infected_df = infected_df.loc[infected_df.index.isin(top_confirmed_df.index)]

In [9]:
# peak = top_infected_df.loc['China','2/18/20']
peak = top_infected_df.loc['United States','4/12/20']
peak

522325

#### Most recent

In [10]:
def recent_stat(country_name, df):
    lookup = list(df.loc[df.index == country_name].values[0])
    lookup.reverse()
    most_recent = next((i for i in lookup if i.any()), None)
    return most_recent

# print(recent_stat("United States", top_infected_df))
# print(recent_stat("Italy", top_infected_df))

#### All Cases

In [11]:
countries = confirmed_df.index
all_df = pd.DataFrame(columns=['Country_Region', 'Confirmed', 'Recovered', 'Fatalities'], data=[])
for country in countries:
    all_df = all_df.append({'Country_Region': country,
                            'Confirmed': recent_stat(country, confirmed_df),
                            'Recovered': recent_stat(country, recovered_df),
                            'Fatalities': recent_stat(country, fatalities_df),
                           }, ignore_index=True)

In [12]:
# First Day Values
def first_day(df, country):
    row = list(df.loc[df.index == country].values[0])
    col = next((i for i, j in enumerate(row) if j.any()), None)     
    first_day = df.columns[col]
    return first_day

# Example:
# print(first_day(top_confirmed_df, "Italy"))
# print(first_day(top_fatalities_df, "Italy"))

In [13]:
# First Day column
firsts = [] 
for country in countries:
    firsts.append(first_day(confirmed_df, country))
all_df['First Day'] = firsts

In [14]:
# Global row
all_df = all_df.append(all_df.sum(), ignore_index=True)
all_df = all_df.replace(all_df.iloc[-1,0], 'Global')
all_df.replace(all_df.iloc[-1, -1], all_df['First Day'].values.min(), inplace=True)
all_df.tail()

,Country_Region,Confirmed,Recovered,Fatalities,First Day
182,Yemen,1,1.0,NaN,4/10/20
183,Zambia,84,37.0,3.0,3/18/20
184,Zimbabwe,31,2.0,4.0,3/20/20
185,Global,2896744,817838.0,202844.0,1/22/20
186,Global,5793488,1635676.0,405688.0,1/22/20


In [15]:
# Save JSON
all_df.to_json('all.json', orient='records')

## Reorganizing Data to First Day of Confirmed Case

In [36]:
# New dataframes with all non-zero values at beginning
def first_day_df(df):
    dates = df.columns.tolist()
    days = range(len(dates))
    
    new_df = pd.DataFrame(columns=days)

    countries = df.index              # list of countries
    for country in countries:
        # Grab the country's row
        row = list(df.loc[df.index == country].values[0])
        # Grab the column index of first non-zero value
        col = next((i for i, j in enumerate(row) if j.any()), None)     
        # Calculate date
        first_day = df.columns[col]
        # Reset first day
        row = row[col:] + row[:col]
        # New rearranged dataframe
        new_df = new_df.append([row])

    # Same countries indexed
    new_df.index = df.index
    new_df = new_df.replace(0,np.NaN)
    
    return new_df

In [37]:
first_confirmed_top = first_day_df(top_confirmed_df)
first_infected_top = first_day_df(top_infected_df)

first_confirmed_all = first_day_df(confirmed_df)
first_fatalities_all = first_day_df(fatalities_df)
# first_recovered_all = first_day_df(recovered_df)
# first_infected_all = first_day_df(infected_df)

#### By Region

In [38]:
# regions_df = pd.read_csv('../demographics/regions.csv')
# confirmed_byregion = pd.merge(regions_df, all_df, on='Country_Region', how='inner')
# confirmed_byregion.sort_values('First Day', inplace=True)
# confirmed_byregion.head(40)

In [39]:
# centralasia = confirmed_byregion.loc[confirmed_byregion['Region'].str.contains('Central Asia')]
# centralasia.sort_values('First Day')

## Fatality Rate

In [40]:
confirmed_df2 = clean_data('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
fatalities_df2 = clean_data('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
fatality_rate_df = fatalities_df2/confirmed_df2
fatality_rate_df = fatality_rate_df.replace(np.nan, 0)
fatality_rate_df = first_day_df(fatality_rate_df)

pd.set_option('display.max_rows', 190)
fatality_rate_df

,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,167
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.025000,0.025000,0.013514,0.023810,0.042553,0.036364,0.036364,0.033333,0.023529,0.022989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,0.083333,0.043478,0.030303,0.026316,0.023810,0.019608,0.018182,0.033898,0.031250,0.028571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Algeria,0.041667,0.076923,0.081081,0.083333,0.074074,0.066667,0.094595,0.103448,0.122222,0.107914,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,0.008850,0.007519,0.006098,0.005319,0.013393,0.011236,0.009740,0.017964,0.021622,0.031915,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,0.285714,0.285714,0.285714,0.250000,0.250000,0.250000,0.200000,0.142857,0.125000,0.117647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Antigua and Barbuda,0.052632,0.105263,0.105263,0.105263,0.095238,0.095238,0.086957,0.086957,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Argentina,0.083333,0.083333,0.058824,0.052632,0.052632,0.064516,0.058824,0.044444,0.035714,0.029412,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Armenia,0.003448,0.003040,0.002457,0.007075,0.006224,0.005639,0.007005,0.010558,0.009511,0.009091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Australia,0.037037,0.033333,0.025641,0.038462,0.036364,0.033333,0.031746,0.039474,0.032967,0.028037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
